In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DIRECTORY = '/content/drive/My Drive/Informatics/Sphere@mail.ru/1_ML/project/'

Imports

In [ ]:
!pip install pymorphy2

In [ ]:
import numpy as np
import pandas as pd
import time

import warnings
warnings.filterwarnings("ignore") 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
import re
import pymorphy2
from itertools import product
from tqdm.notebook import tqdm
from copy import deepcopy
from collections import Counter

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

# Данные

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stop = '(' + '|'.join(stopwords.words('english') + stopwords.words('russian') + ['u']) + ')'
stemmerR = SnowballStemmer("russian")
stemmerE = SnowballStemmer("english")
def make_standart_str(s):
    s = s.lower()
    
    s = re.sub('\[архив\]', '', s)
    s = re.sub('страница\s\d+', '', s)
    s = re.sub('\S+\.[com,ru,org,net]', '', s)
    s = re.sub('[^\w\s]', '', s)

    s = re.sub('\s+', ' ', s)
    s = ' '.join([morph.parse(word)[0].normal_form for word in s.split()])

    s = ' ' + s + ' '
    s = re.sub(f'[^\d\w]{stop}[^\d\w]', ' ', s)
    s = re.sub('\s+', ' ', s)

    s = ' '.join([stemmerR.stem(stemmerE.stem(word)) for word in s.split()])
    return s.strip()

In [ ]:
def add_h1_h6(content):
    soup = BeautifulSoup(content)
    title = ' '.join(map(lambda x: make_standart_str(x.text), soup.find_all('title')))
    text = ' '.join(map(lambda x: make_standart_str(x.text), soup.find_all(re.compile('^h[1-6]$'))))
    title += ' ' + text
    title = re.sub('\s+', ' ', title)
    title = ' '.join(set(title.split()))
    return title

In [ ]:
def get_doc_to_title():
    counter = Counter()
    doc_to_title = {}
    with open(DIRECTORY + 'docs_titles.tsv') as f:
        for num_line, line in tqdm(enumerate(f), total=28027):
            if num_line == 0:
                continue
            data = line.strip().split('\t', 1)
            doc_id = int(data[0])
            if len(data) == 1:
                title = ''
            else:
                title = make_standart_str(data[1])
                title = ' '.join(set(title.split()))
            doc_to_title[doc_id] = title
            for word in title.split():
                counter[word] += 1
    return doc_to_title, counter

doc_to_title, title_counter = get_doc_to_title()
# title_counter.most_common(10)

In [ ]:
import tarfile
import json
import os
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool

In [ ]:
def process_page(filename):
    with open('content/' + filename, encoding='utf-8') as f:
        content_id = int(filename.split('.')[0])

        doc_to_url[content_id] = f.readline().strip()
        content = f.read()

        try:
            doc_to_content[content_id] = add_h1_h6(content)
            for word in doc_to_content[content_id].split():
                counter[word] += 1
        except Exception as e:
            with lock:
                print(e)
                print(content_id)
                print()
        with lock:
            pbar.update(1)

In [ ]:
# %%time

# tar = tarfile.open(DIRECTORY + "content.tar.gz", "r:gz")
# tar.extractall()
# tar.close()

In [ ]:
# %%time

# docs_data = pd.DataFrame(columns=['doc_id', 'title', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'url'])
# for filename in tqdm(files):
#     with open('content/' + filename, encoding='utf-8') as f:
#         doc_id = int(filename.split('.')[0])

#         url = f.readline().strip()
#         content = f.read()

#         soup = BeautifulSoup(content)
#         title = (' '.join(map(lambda x: x.text, soup.find_all('title')))).strip()
#         h = []
#         for hnum in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
#             h.append((' '.join(map(lambda x: x.text, soup.find_all(hnum)))).strip())
        
#         df = pd.DataFrame({
#             'doc_id': doc_id,
#             'url': url,
#             'title': title,
#             'h1': [h[0]],
#             'h2': [h[1]],
#             'h3': [h[2]],
#             'h4': [h[3]],
#             'h5': [h[4]],
#             'h6': [h[5]]
#         })

#         docs_data = pd.concat([docs_data, df])

In [ ]:
# %%time

# files = os.listdir('content')

# doc_to_content = {}
# doc_to_url = {}
# counter = Counter()

# with ThreadPool(processes=8) as pool, tqdm(total=len(files)) as pbar:
#     lock = pbar.get_lock()
#     pool.map(process_page, files)

# pool.join()

# counter.most_common(10)

In [ ]:
# df = pd.DataFrame({
#     'doc_id': doc_to_content.keys(),
#     'titleh16': doc_to_content.values()
# })

# print(df.shape)

# with open('doc_to_titleh16.csv', 'w') as f:
#     f.write(df.to_csv(index=False))

In [ ]:
doc_to_titleh16 = pd.read_csv('doc_to_titleh16.csv').set_index('doc_id').fillna('').to_dict()['titleh16']

In [ ]:
! pip install fasttext

     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3039189 sha256=b6db6377e52b6db844d2fb39df89e51d596e55e460ad08d69824c32b247eb0d0
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
# import fasttext.util
# fasttext.util.download_model('ru', if_exists='ignore', dimension=10)

In [ ]:
# %%time

# import gzip
# import shutil
# with gzip.open(DIRECTORY + 'cc.ru.300.bin.gz', 'rb') as f_in:
#     with open('cc.ru.300.bin', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

CPU times: user 52.1 s, sys: 12.5 s, total: 1min 4s
Wall time: 2min 33s


In [ ]:
# doc_to_title = doc_to_titleh16

# FUNCS

In [ ]:
%%time

# ft = fasttext.load_model('cc.ru.300.bin')
ft = None

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [ ]:
def get_data_titledata(doc_to_title, test=False):
    name = 'test' if test else 'train'

    data = pd.read_csv(DIRECTORY + name + '_groups.csv')
    groups_titledata = {}
    # groups_words = {}

    max_groups_titledata_len = 0
    # max_groups_words_len = 0

    for i in tqdm(range(len(data))):
        new_doc = data.iloc[i]
        doc_group = new_doc['group_id']
        doc_id = new_doc['doc_id']
        if not test:
            target = new_doc['target']
        title = doc_to_title[doc_id]
        if doc_group not in groups_titledata:
            groups_titledata[doc_group] = []
            # groups_words[doc_group] = set()
        to_append = (doc_id, title) if test else (doc_id, title, target)

        groups_titledata[doc_group].append(to_append)
        if len(groups_titledata[doc_group]) > max_groups_titledata_len:
            max_groups_titledata_len = len(groups_titledata[doc_group])

        # groups_words[doc_group] = groups_words[doc_group] | set(title.split())
        # if len(groups_words[doc_group]) > max_groups_words_len:
        #     max_groups_words_len = len(groups_words[doc_group])
    
    print(f'max_groups_titledata_len = {max_groups_titledata_len}')
    #print(f'max_groups_words_len = {max_groups_words_len}')

    return data, groups_titledata#, groups_words


def get_x_y_data(test=False):
    name = 'test' if test else 'train'

    data, groups_titledata = get_data_titledata(doc_to_title, test)
    max_group_len_all_dist = title_n_features_all_dist
    # max_group_len_fasttext = title_n_features_fasttext
    max_group_len_tfidf = title_n_features_tfidf
    # max_group_word_count = 766  # TODO
    # fasttext_simm_groups = fasttext_simm[name]
    all_dist_groups = all_dist_mas[name]
    tfidf_vector_groups = tfidf_vector_all[name]

    y = []
    X = []
    groups = []
    for new_group in tqdm(groups_titledata):
        docs = groups_titledata[new_group]

        for k, tpl in enumerate(docs):
            if test:
                doc_id, title = tpl
            else:
                doc_id, title, target_id = tpl
                y.append(target_id)
            groups.append(new_group)

            all_dist = all_dist_groups[doc_id][:max_group_len_all_dist]
            # simm = fasttext_simm_groups[doc_id][:max_group_len_fasttext]
            tfidf_vec = tfidf_vector_groups[doc_id][:max_group_len_tfidf]

            features = []
            features.extend(all_dist)
            # features.extend(simm)
            features.extend(tfidf_vec)
            
            # for word in groups_words[new_group]:
            #     features.append(1 if word in words else 0)
            
            # all_dist_len = len(all_dist)

            # while len(features) < all_dist_len + max_group_word_count:
            #     features.append(0)

            X.append(features)

    X = np.array(X)
    y = np.array(y)
    groups = np.array(groups)
    print (X.shape, y.shape, groups.shape)
    return X, y, groups, data

In [ ]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer

def get_tfidf(test=False):
    name = 'test' if test else 'train'

    data, groups_titledata = get_data_titledata(doc_to_title, test)
    max_group_len = 120

    tfidf_vector_all = {}

    tfidf_vectorizers = {}
    for new_group in tqdm(groups_titledata):
        docs = groups_titledata[new_group]

        for k, tpl in enumerate(docs):
            if test:
                doc_id, title = tpl
            else:
                doc_id, title, target_id = tpl

            tfidf_vector = []
            vectorizer = TfidfVectorizer(max_features=120)
            vectorizer.fit([title, 'hh'])
            for j in range(0, len(docs)):
                if k == j:
                    continue
                if test:
                    doc_id_j, title_j = docs[j]
                else:
                    doc_id_j, title_j, target_j = docs[j]
                
                tfidf_vector.append(vectorizer.transform([title_j]).toarray().sum())

            tfidf_vector = sorted(tfidf_vector, reverse=True)[:max_group_len]
            for i in range(len(tfidf_vector), max_group_len):
                tfidf_vector.append(0)

            features = []
            features.extend(tfidf_vector)

            tfidf_vector_all[doc_id] = features

    return tfidf_vector_all

tfidf_vector_all = {
    'train': get_tfidf(),
    'test': get_tfidf(test=True)
}
print('tfidf_vector_all size =', len(tfidf_vector_all['train']))


max_groups_titledata_len = 108



max_groups_titledata_len = 120



tfidf_vector_all size = 11648
CPU times: user 28min 15s, sys: 20.7 s, total: 28min 36s
Wall time: 28min 18s


In [ ]:
# %%time

# from sklearn.metrics.pairwise import cosine_similarity

# def get_simm(test=False):
#     name = 'test' if test else 'train'

#     data, groups_titledata = get_data_titledata(doc_to_title, test)
#     max_group_len = 120

#     fasttext_simm = {}
#     for new_group in tqdm(groups_titledata):
#         docs = groups_titledata[new_group]

#         for k, tpl in enumerate(docs):
#             if test:
#                 doc_id, title = tpl
#             else:
#                 doc_id, title, target_id = tpl

#             fasttext_vectors = []
#             for j in range(0, len(docs)):
#                 if k == j:
#                     continue
#                 if test:
#                     doc_id_j, title_j = docs[j]
#                 else:
#                     doc_id_j, title_j, target_j = docs[j]
                
#                 fasttext_j = ft.get_sentence_vector(title_j)
#                 fasttext_vectors.append(fasttext_j)

#             simm = list(cosine_similarity([ft.get_sentence_vector(title)], fasttext_vectors)[0])
#             for i in range(len(simm), max_group_len):
#                 simm.append(0)
                
#             simm = sorted(simm, reverse=True)[:max_group_len]

#             features = []
#             features.extend(simm)

#             fasttext_simm[doc_id] = features

#     return fasttext_simm

# fasttext_simm = {
#     'train': get_simm(),
#     'test': get_simm(test=True)
# }
# print('fasttext_simm size =', len(fasttext_simm['train']))


max_groups_titledata_len = 108



max_groups_titledata_len = 120



fasttext_simm size = 11648
CPU times: user 2min 50s, sys: 2min 10s, total: 5min 1s
Wall time: 2min 38s


In [ ]:
%%time

def get_all_dist_mas(test=False):
    name = 'test' if test else 'train'

    data, groups_titledata = get_data_titledata(doc_to_title, test)
    max_group_len = 120

    all_dist_mas = {}

    for new_group in tqdm(groups_titledata):
        docs = groups_titledata[new_group]

        for k, tpl in enumerate(docs):
            if test:
                doc_id, title = tpl
            else:
                doc_id, title, target_id = tpl

            all_dist = []
            words = set(title.strip().split())
            for j in range(0, len(docs)):
                if k == j:
                    continue
                if test:
                    doc_id_j, title_j = docs[j]
                else:
                    doc_id_j, title_j, target_j = docs[j]
                
                words_j = set(title_j.strip().split())
                all_dist.append(len(words.intersection(words_j)))

            for i in range(len(all_dist), max_group_len):
                all_dist.append(0)
            all_dist = sorted(all_dist, reverse=True)[:max_group_len]

            features = []
            features.extend(all_dist)
            
            all_dist_mas[doc_id] = features

    return all_dist_mas

all_dist_mas = {
    'train': get_all_dist_mas(),
    'test': get_all_dist_mas(test=True)
}
print('all_dist_mas size =', len(all_dist_mas['train']))


max_groups_titledata_len = 108



max_groups_titledata_len = 120



all_dist_mas size = 11648
CPU times: user 9.65 s, sys: 164 ms, total: 9.81 s
Wall time: 9.9 s


In [ ]:
def cross_validate(model, fit_params=None):
    return cross_val_score(model, X_train, y_train, cv=3,
                           groups=groups_train, scoring='f1',
                           fit_params=fit_params).mean()

In [ ]:
def search_parametrs(clf, parametrs, X_train, y_train, fit_params=None, print_scores=False):
    best_model = None
    best_score = 0
    best_params = None
    model = deepcopy(clf)
    keys = parametrs.keys()
    for cur_param in tqdm(list(product(*parametrs.values()))):
        params_to_set = dict()
        for i, key in enumerate(keys):
            params_to_set[key] = cur_param[i]
        model.set_params(**params_to_set)
        try:
            score = cross_validate(model, fit_params=fit_params)
        except Exception as e:
            print("ERROR:\n", str(e))
            continue
        if print_scores:
            print(f'{str(params_to_set)} - SCORE: {score}')
        if score > best_score:
            best_score = score
            best_model = deepcopy(model)
            best_params = params_to_set
    print(best_params)
    print('F1-Score: ', best_score)
    print(best_model)
    return best_model

In [ ]:
def make_precision(model, X_test, test_data, to_drive=False, y_test=None):
    if y_test is None:
        y_test = np.asarray(model.predict(X_test))
    y_test = y_test.reshape((-1))
    y_test = np.asarray(y_test, dtype=np.int64)
    assert y_test.shape[0] == X_test.shape[0]
    print(X_test.shape)
    test_data["target"] = pd.Series(y_test)
    name = str(time.time()) + "_predict.csv"
    if to_drive:
        name = DIRECTORY + 'predictions/' + name
    with open(name, "w") as f:
        f.write(test_data.to_csv(columns=("pair_id", "target"), index=False))

    print(name)
    print(model)

    return y_test

In [ ]:
def add_features(X, feat):
    return np.hstack((X, feat))

In [ ]:
def update_features(by_model):
    return add_features(X_train, models_data[by_model]['pred_train']), add_features(X_test, models_data[by_model]['pred_test'])

In [ ]:
def union_precision(keys, train=False):
    pred_key = 'pred_train' if train else 'pred_test'
    concat = models_data[keys[0]][pred_key]
    print(models_data[keys[0]]['model'])
    for by_model in keys[1:]:
        pred = models_data[by_model][pred_key]
        print(models_data[by_model]['model'])
        concat = np.hstack((concat, pred))
    return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=1, arr=concat).reshape((-1, 1))

In [ ]:
from sklearn.metrics import f1_score

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1 - f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [ ]:
def update_models_data(model_name):
    global models_data
    models_data[model_name]['cv_f1_score'] = cross_validate(models_data[model_name]['model'])

    models_data[model_name]['model'].fit(X_train, y_train)
    models_data[model_name]['pred_train'] = models_data[model_name]['model'].predict(X_train).reshape((-1, 1))
    models_data[model_name]['pred_test'] = models_data[model_name]['model'].predict(X_test).reshape((-1, 1))
    models_data[model_name]['pred_proba_train'] = models_data[model_name]['model'].predict_proba(X_train)[:, 1].reshape((-1, 1))
    models_data[model_name]['pred_proba_test'] = models_data[model_name]['model'].predict_proba(X_test)[:, 1].reshape((-1, 1))

In [ ]:
def union_precision_proba(keys, train=False, th=0.5, printData=True):
    pred_key = 'pred_proba_train' if train else 'pred_proba_test'
    concat = models_data[keys[0]][pred_key]
    if printData:
        print(f'th={th}')
        print(models_data[keys[0]]['model'])
    for by_model in keys[1:]:
        pred = models_data[by_model][pred_key]
        if printData:
            print(models_data[by_model]['model'])
        concat = np.hstack((concat, pred))
    return np.array(np.mean(concat, axis=1) > th, dtype=int).reshape((-1, 1))

In [ ]:
def search_best_th(model_names):
    best_score = 0
    best_th = 0.5

    for th in np.linspace(0.01, 1, 100):
        score = f1_score(y_train, union_precision_proba(model_names, train=True, th=th, printData=False))
        if score > best_score:
            #print(th, score)
            best_score = score
            best_th = th
    return best_th, best_score

# Load data

In [ ]:
title_n_features_all_dist = 77
title_n_features_fasttext = 0
title_n_features_tfidf = 120

In [ ]:
X_train, y_train, groups_train, train_data = get_x_y_data()
X_test, y_test, groups_test, test_data = get_x_y_data(test=True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


max_groups_titledata_len = 108



(11690, 197) (11690,) (11690,)



max_groups_titledata_len = 120



(16627, 197) (0,) (16627,)


In [ ]:
# TODO: Blending, Stacking on Boosting
# TODO: TFIDF, DBSCAN

# Models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
from collections import defaultdict
models_data = defaultdict(lambda: {
        'model': None,
        'pred_train': None,
        'pred_test': None,
        'cv_f1_score': None,
    })

#### KNeighborsClassifier

In [ ]:
%%time

model_name = 'KNC'
parametrs = {
    'n_neighbors': np.linspace(10, 400, 20, dtype=int),
    'metric': ['cosine'],
    'algorithm': ['brute'],
}


# models_data[model_name]['model'] = KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
#                      metric_params=None, n_jobs=None, n_neighbors=135, p=2,
#                      weights='uniform')
# print(cross_validate(models_data[model_name]['model']))

models_data[model_name]['model'] = search_parametrs(KNeighborsClassifier(), parametrs, X_train, y_train)

clf = models_data[model_name]['model']
parametrs = {
    'n_neighbors': np.linspace(clf.n_neighbors - 20, clf.n_neighbors + 20, 41, dtype=int),
}
models_data[model_name]['model'] = search_parametrs(clf, parametrs, X_train, y_train, print_scores=False)

update_models_data(model_name)

print(search_best_th([model_name]))


{'n_neighbors': 235, 'metric': 'cosine', 'algorithm': 'brute'}
F1-Score:  0.6507145315357302
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=235, p=2,
                     weights='uniform')



{'n_neighbors': 229}
F1-Score:  0.6529660810630926
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=229, p=2,
                     weights='uniform')


#### SVC

In [ ]:
%%time
model_name = 'SVC'
parametrs = {
    'C': np.linspace(0.1, 1, 10),
    'kernel': ['rbf', 'sigmoid', 'linear', 'poly'],
    'max_iter': [50],
    'gamma': ['auto', 'scale']
}

models_data[model_name]['model'] = search_parametrs(SVC(random_state=77, probability=True), parametrs, X_train, y_train)

# models_data[model_name]['model'] = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
#                        max_depth=1, max_features='auto', max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=6,
#                        min_weight_fraction_leaf=0.0, presort='deprecated',
#                        random_state=77, splitter='best')
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))


{'C': 0.1, 'kernel': 'sigmoid', 'max_iter': 100, 'gamma': 'scale'}
F1-Score:  0.6671611704010781
SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=100, probability=True, random_state=77, shrinking=True, tol=0.001,
    verbose=False)
CPU times: user 7min 25s, sys: 219 ms, total: 7min 26s
Wall time: 7min 26s


#### DecisionTreeClassifier

In [ ]:
%%time

model_name = 'DTC'
parametrs = {
    'max_depth': np.linspace(1, 50, 5, dtype=int),
    'min_samples_leaf': np.linspace(1, 50, 5, dtype=int),
    'min_samples_split': np.linspace(1, 50, 5, dtype=int),
    'max_features': ['auto'],
    'criterion': ['gini', 'entropy']
}

models_data[model_name]['model'] = search_parametrs(DecisionTreeClassifier(random_state=77), parametrs, X_train, y_train)

# models_data[model_name]['model'] = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
#                        max_depth=1, max_features='auto', max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=6,
#                        min_weight_fraction_leaf=0.0, presort='deprecated',
#                        random_state=77, splitter='best')
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))


{'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 13, 'max_features': 'auto', 'criterion': 'entropy'}
F1-Score:  0.6757518310521097
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=1, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=13,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=77, splitter='best')
(0.09999999999999999, 0.6834131814061893)
CPU times: user 56.3 s, sys: 678 ms, total: 57 s
Wall time: 57.1 s


#### RandomForestClassifier

In [ ]:
%%time

model_name = 'RFC'
parametrs = {
    'n_estimators': [100],
    'max_depth': np.linspace(1, 100, 5, dtype=int),
    'min_samples_leaf': range(1, 8),
    'min_samples_split': range(2, 10, 2)
}

models_data[model_name]['model'] = search_parametrs(RandomForestClassifier(random_state=77), parametrs, X_train, y_train)

# models_data[model_name]['model'] = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='gini', max_depth=25, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=6, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=1000,
#                        n_jobs=None, oob_score=False, random_state=77, verbose=0,
#                        warm_start=False)
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

#### ExtraTreesClassifier

In [ ]:
%%time

model_name = 'ETC'
parametrs = {
    'n_estimators': [100],
    'max_depth': np.linspace(1, 100, 5, dtype=int),
    'min_samples_leaf': range(1, 8),
    'min_samples_split': range(2, 10, 2)
}

models_data[model_name]['model'] = search_parametrs(ExtraTreesClassifier(random_state=77), parametrs, X_train, y_train)

# models_data[model_name]['model'] = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
#                        criterion='gini', max_depth=1, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=6, min_samples_split=4,
#                        min_weight_fraction_leaf=0.0, n_estimators=1000,
#                        n_jobs=None, oob_score=False, random_state=77, verbose=0,
#                        warm_start=False)
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

#### AdaBoostClassifier

In [ ]:
%%time

model_name = 'ABC'
parametrs = {
    'n_estimators': np.linspace(1, 50, 5, dtype=int),
    'learning_rate': np.linspace(1e-7, 1, 21),
    'algorithm': ['SAMME.R', 'SAMME']
}

models_data[model_name]['model'] = search_parametrs(AdaBoostClassifier(random_state=77), parametrs, X_train, y_train, print_scores=False)

# models_data[model_name]['model'] = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
#                    learning_rate=1e-07, n_estimators=1, random_state=77)
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

KeyboardInterrupt: ignored

#### SGDClassifier

In [ ]:
%%time

model_name = 'SGD'
parametrs = {
    'loss': ['log', 'modified_huber', 'huber'],
    'penalty': ['elasticnet'],
    'l1_ratio': np.linspace(0, 1, 5),
    'alpha': np.linspace(1e-4, 1e-1, 5),
    'max_iter': np.linspace(1e2, 1e4, 5, dtype=int)
}

models_data[model_name]['model'] = search_parametrs(SGDClassifier(random_state=77), parametrs, X_train, y_train)

# models_data[model_name]['model'] = SGDClassifier(alpha=1e-05, average=False, class_weight=None,
#               early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
#               l1_ratio=0.0, learning_rate='optimal', loss='hinge', max_iter=100,
#               n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
#               power_t=0.5, random_state=35, shuffle=True, tol=0.001,
#               validation_fraction=0.1, verbose=0, warm_start=False)
# print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))


{'loss': 'modified_huber', 'penalty': 'elasticnet', 'l1_ratio': 0.5, 'alpha': 0.050050000000000004, 'max_iter': 100}
F1-Score:  0.6583389589308942
SGDClassifier(alpha=0.050050000000000004, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.5, learning_rate='optimal', loss='modified_huber',
              max_iter=100, n_iter_no_change=5, n_jobs=None,
              penalty='elasticnet', power_t=0.5, random_state=77, shuffle=True,
              tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)
(0.33, 0.7252747252747254)
CPU times: user 8min 51s, sys: 1min 56s, total: 10min 47s
Wall time: 8min 46s


#### Boosting

In [ ]:
def boosting_search(clf, model_name):
    parametrs = {
        'learning_rate': np.linspace(1e-2, 1, 100),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=False)

    parametrs = {
        'max_depth': np.linspace(1, 15, 6, dtype=int),
        'scale_pos_weight': np.linspace(2, 3, 5),
        'max_leaves': np.linspace(0, 10, 5, dtype=int),
    }
    if model_name != 'CBC':
        parametrs['min_child_weight'] = np.linspace(0, 100, 5)
    clf = search_parametrs(clf, parametrs, X_train, y_train)
    cur_scale = clf.get_param('scale_pos_weight') if model_name == 'CBC' else clf.scale_pos_weight

    parametrs = {
        'scale_pos_weight': np.linspace(cur_scale - 0.01, cur_scale + 0.01, 21),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=True)

    cur_scale = clf.get_param('scale_pos_weight') if model_name == 'CBC' else clf.scale_pos_weight
    parametrs = {
        'scale_pos_weight': np.linspace(cur_scale - 0.001, cur_scale + 0.001, 21),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=True)

    parametrs = {
        'subsample': np.linspace(1e-2, 1, 100),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train)

    parametrs = {
        'reg_lambda': np.linspace(0, 5, 11),
    }
    if model_name != 'CBC':
        parametrs['reg_alpha'] = np.linspace(0, 5, 11)
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=False)

    cur_lr = clf.get_param('learning_rate') if model_name == 'CBC' else clf.learning_rate
    parametrs = {
        'learning_rate': np.linspace(1e-5, cur_lr, 10),
        'n_estimators': [25, 50, 100]
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=True)

    return clf

In [ ]:
def boosting_search_lite(clf, model_name):
    parametrs = {
        'learning_rate': np.linspace(1e-2, 1, 10),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=False)

    parametrs = {
        'max_depth': np.linspace(1, 10, 3, dtype=int),
        'scale_pos_weight': np.linspace(2, 3, 5),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train)
    cur_scale = clf.get_param('scale_pos_weight') if model_name == 'CBC' else clf.scale_pos_weight

    parametrs = {
        'scale_pos_weight': np.linspace(cur_scale - 0.01, cur_scale + 0.01, 21),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=True)

    parametrs = {
        'subsample': np.linspace(1e-2, 1, 10),
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train)

    parametrs = {
        'reg_lambda': np.linspace(0, 5, 6),
    }
    if model_name != 'CBC':
        parametrs['reg_alpha'] = np.linspace(0, 5, 6)
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=False)

    cur_lr = clf.get_param('learning_rate') if model_name == 'CBC' else clf.learning_rate
    parametrs = {
        'learning_rate': np.linspace(1e-5, cur_lr, 10),
        'n_estimators': [25, 50, 100]
    }
    clf = search_parametrs(clf, parametrs, X_train, y_train, print_scores=True)

    return clf

#### XGBClassifier

In [ ]:
%%time

clf = XGBClassifier(random_state=77, n_estimators=25, objective='binary:logistic', eval_metric='rmse')
clf.fit(X_train, y_train)

CPU times: user 2.74 s, sys: 4.74 ms, total: 2.75 s
Wall time: 2.76 s


In [ ]:
%%time

clf = boosting_search_lite(clf, 'XBC')


{'learning_rate': 0.12}
F1-Score:  0.6896835165497824
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
              gamma=0, learning_rate=0.12, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=77,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)



{'max_depth': 3, 'scale_pos_weight': 2.25}
F1-Score:  0.7146895701127
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
              gamma=0, learning_rate=0.12, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=25, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=77,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=2.25, seed=None,
              silent=None, subsample=1, verbosity=1)


{'scale_pos_weight': 2.24} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.241} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.242} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2430000000000003} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.244} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.245} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.246} - SCORE: 0.7153619520767704
{'scale_pos_weight': 2.247} - SCORE: 0.7153619520767704
{'scale_pos_weight': 2.248} - SCORE: 0.7147919385738281
{'scale_pos_weight': 2.249} - SCORE: 0.7147919385738281
{'scale_pos_weight': 2.25} - SCORE: 0.7146895701127
{'scale_pos_weight': 2.251} - SCORE: 0.7133456397957282
{'scale_pos_weight': 2.252} - SCORE: 0.7137215796453522
{'scale_pos_weight': 2.253} - SCORE: 0.7098720995006023
{'scale_pos_weight': 2.254} - SCORE: 0.7098720995006023
{'scale_pos_weight': 2.255} - SCORE: 0.7098720995006023
{'scale_pos_weight': 2.256} - SCORE: 0.7098720995006023
{'scale_pos_weight': 2.2569999999999997}

{'scale_pos_weight': 2.2390000000000003} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2391000000000005} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2392000000000003} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2393} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2394000000000003} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2395000000000005} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2396000000000003} - SCORE: 0.7156478764196624
{'scale_pos_weight': 2.2397} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2398000000000002} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2399000000000004} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.24} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2401} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2402} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2403000000000004} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2404} - SCORE: 0.7155567989318946
{'scale_pos_weight': 2.2405} - SCORE: 0

KeyboardInterrupt: ignored

In [ ]:
model_name = 'XBC'

# models_data[model_name]['model'] = deepcopy(clf)

models_data[model_name]['model'] = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
              gamma=0, learning_rate=0.17, max_delta_step=0, max_depth=12,
              max_leaves=0, min_child_weight=100.0, missing=None,
              n_estimators=25, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=77, reg_alpha=2.0,
              reg_lambda=1.0, scale_pos_weight=2.2464, seed=None, silent=None,
              subsample=0.66, verbosity=1)
print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

0.7287719635364217
(0.55, 0.7653146853146854)


#### LGBMClassifier

In [ ]:
%%time

clf = LGBMClassifier(random_state=77, n_estimators=25, objective='binary', matric='auc')
clf.fit(X_train, y_train)

CPU times: user 1.39 s, sys: 13.6 ms, total: 1.4 s
Wall time: 1.44 s


In [ ]:
%%time

clf = boosting_search(clf, 'LBC')


{'learning_rate': 0.18000000000000002}
F1-Score:  0.637576588727522
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.18000000000000002,
               matric='auc', max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=25,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)



{'max_depth': 3, 'min_child_weight': 100.0, 'scale_pos_weight': 2.0, 'max_leaves': 0}
F1-Score:  0.6999687262972508
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.18000000000000002,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0, min_split_gain=0.0, n_estimators=25,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=0.0, reg_lambda=0.0, scale_pos_weight=2.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


{'scale_pos_weight': 1.99} - SCORE: 0.7003502086667353
{'scale_pos_weight': 1.9909999999999999} - SCORE: 0.700258273344076
{'scale_pos_weight': 1.992} - SCORE: 0.700258273344076
{'scale_pos_weight': 1.9929999999999999} - SCORE: 0.6995097130931716
{'scale_pos_weight': 1.994} - SCORE: 0.6995097130931716
{'scale_pos_weight': 1.9949999999999999} - SCORE: 0.6995097130931716
{'scale_pos_weight': 1.996} - SCORE: 0.6998577610262102
{'scale_pos_weight': 1.9969999999999999} - SCORE: 0.6995834710656599
{'scale_pos_weight': 1.998} - SCORE: 0.6995834710656599
{'scale_pos_weight': 1.9989999999999999} - SCORE: 0.6999687262972508
{'scale_pos_weight': 2.0} - SCORE: 0.6999687262972508
{'scale_pos_weight': 2.001} - SCORE: 0.6992520180319391
{'scale_pos_weight': 2.002} - SCORE: 0.6998652298726765
{'scale_pos_weight': 2.0029999999999997} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.004} - SCORE: 0.7004053922479656
{'scale_pos_weight': 2.005} - SCORE: 0.7003126417567138
{'scale_pos_weight': 2.006} - S

{'scale_pos_weight': 2.002} - SCORE: 0.6998652298726765
{'scale_pos_weight': 2.0020999999999995} - SCORE: 0.6998652298726765
{'scale_pos_weight': 2.0021999999999998} - SCORE: 0.6998652298726765
{'scale_pos_weight': 2.0023} - SCORE: 0.6998652298726765
{'scale_pos_weight': 2.0023999999999997} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0024999999999995} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0025999999999997} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0027} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0027999999999997} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0028999999999995} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0029999999999997} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0031} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0031999999999996} - SCORE: 0.7007652815175341
{'scale_pos_weight': 2.0032999999999994} - SCORE: 0.7006761886279652
{'scale_pos_weight': 2.0033999999999996} - SCORE: 0.7006761886279652
{'scale_pos_we


{'subsample': 0.01}
F1-Score:  0.7007652815175341
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.18000000000000002,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0, min_split_gain=0.0, n_estimators=25,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=0.0, reg_lambda=0.0,
               scale_pos_weight=2.0023999999999997, silent=True, subsample=0.01,
               subsample_for_bin=200000, subsample_freq=0)



{'reg_lambda': 0.0, 'reg_alpha': 1.0}
F1-Score:  0.7010809988416741
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.18000000000000002,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0, min_split_gain=0.0, n_estimators=25,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=1.0, reg_lambda=0.0,
               scale_pos_weight=2.0023999999999997, silent=True, subsample=0.01,
               subsample_for_bin=200000, subsample_freq=0)


{'learning_rate': 1e-05, 'n_estimators': 25} - SCORE: 0.0
{'learning_rate': 1e-05, 'n_estimators': 50} - SCORE: 0.0
{'learning_rate': 1e-05, 'n_estimators': 100} - SCORE: 0.0
{'learning_rate': 0.02000888888888889, 'n_estimators': 25} - SCORE: 0.50536318574757
{'learning_rate': 0.02000888888888889, 'n_estimators': 50} - SCORE: 0.6667919076008858
{'learning_rate': 0.02000888888888889, 'n_estimators': 100} - SCORE: 0.6892587295520868
{'learning_rate': 0.04000777777777778, 'n_estimators': 25} - SCORE: 0.6694277850780717
{'learning_rate': 0.04000777777777778, 'n_estimators': 50} - SCORE: 0.6886426756357729
{'learning_rate': 0.04000777777777778, 'n_estimators': 100} - SCORE: 0.6982277810276029
{'learning_rate': 0.06000666666666667, 'n_estimators': 25} - SCORE: 0.6863062023952001
{'learning_rate': 0.06000666666666667, 'n_estimators': 50} - SCORE: 0.6981930748501148
{'learning_rate': 0.06000666666666667, 'n_estimators': 100} - SCORE: 0.7008462402982886
{'learning_rate': 0.08000555555555555, 'n

In [ ]:
model_name = 'LBC'

# models_data[model_name]['model'] = deepcopy(clf)

models_data[model_name]['model'] = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.08000555555555555,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=1.0, reg_lambda=0.0,
               scale_pos_weight=2.0023999999999997, silent=True, subsample=0.01,
               subsample_for_bin=200000, subsample_freq=0)
print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

0.7285258747786859
(0.51, 0.7606412382531786)


#### CatBoostClassifier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
%%time

clf = CatBoostClassifier(random_state=77, n_estimators=25, objective='Logloss', eval_metric='F1', verbose=0)
#clf.fit(X_train, y_train)

CPU times: user 215 µs, sys: 40 µs, total: 255 µs
Wall time: 262 µs


In [ ]:
%%time

clf = boosting_search(clf, 'CBC')


{'learning_rate': 0.35000000000000003}
F1-Score:  0.6716498202100855



{'max_depth': 1, 'scale_pos_weight': 2.0, 'max_leaves': 2}
F1-Score:  0.6887109640898609


{'scale_pos_weight': 1.99} - SCORE: 0.6914186446656642
{'scale_pos_weight': 1.9909999999999999} - SCORE: 0.6913685143077962
{'scale_pos_weight': 1.992} - SCORE: 0.6883102051409308
{'scale_pos_weight': 1.9929999999999999} - SCORE: 0.688260020472069
{'scale_pos_weight': 1.994} - SCORE: 0.6913991260769169
{'scale_pos_weight': 1.9949999999999999} - SCORE: 0.6897360869895314
{'scale_pos_weight': 1.996} - SCORE: 0.688640472546052
{'scale_pos_weight': 1.9969999999999999} - SCORE: 0.6912354798101394
{'scale_pos_weight': 1.998} - SCORE: 0.6898225241394836
{'scale_pos_weight': 1.9989999999999999} - SCORE: 0.6897231280015772
{'scale_pos_weight': 2.0} - SCORE: 0.6887109640898609
{'scale_pos_weight': 2.001} - SCORE: 0.6908448150301365
{'scale_pos_weight': 2.002} - SCORE: 0.6902537459082875
{'scale_pos_weight': 2.0029999999999997} - SCORE: 0.6861087119787478
{'scale_pos_weight': 2.004} - SCORE: 0.6883056427572729
{'scale_pos_weight': 2.005} - SCORE: 0.690127768187725
{'scale_pos_weight': 2.006} - SC

{'scale_pos_weight': 2.009} - SCORE: 0.6899972261060503
{'scale_pos_weight': 2.0091} - SCORE: 0.6888073373227751
{'scale_pos_weight': 2.0092} - SCORE: 0.6927356078303163
{'scale_pos_weight': 2.0092999999999996} - SCORE: 0.6910337878544812
{'scale_pos_weight': 2.0094} - SCORE: 0.6906920071320689
{'scale_pos_weight': 2.0095} - SCORE: 0.6905022314852686
{'scale_pos_weight': 2.0096} - SCORE: 0.6912813705935222
{'scale_pos_weight': 2.0096999999999996} - SCORE: 0.6926509895268221
{'scale_pos_weight': 2.0098} - SCORE: 0.6924176267814713
{'scale_pos_weight': 2.0099} - SCORE: 0.688721796971946
{'scale_pos_weight': 2.01} - SCORE: 0.6920772340912017
{'scale_pos_weight': 2.0100999999999996} - SCORE: 0.6902638159268228
{'scale_pos_weight': 2.0101999999999998} - SCORE: 0.6904638495363438
{'scale_pos_weight': 2.0103} - SCORE: 0.6912899349004515
{'scale_pos_weight': 2.0103999999999997} - SCORE: 0.6895005145009482
{'scale_pos_weight': 2.0104999999999995} - SCORE: 0.6907208577058275
{'scale_pos_weight':


{'subsample': 0.8}
F1-Score:  0.6927356078303163



{'reg_lambda': 3.0}
F1-Score:  0.6927356078303163


{'learning_rate': 1e-05, 'n_estimators': 25} - SCORE: 0.6869397562191534
{'learning_rate': 1e-05, 'n_estimators': 50} - SCORE: 0.6864266970903302
{'learning_rate': 1e-05, 'n_estimators': 100} - SCORE: 0.6862256559386982
{'learning_rate': 0.03889777777777778, 'n_estimators': 25} - SCORE: 0.6893262382195156
{'learning_rate': 0.03889777777777778, 'n_estimators': 50} - SCORE: 0.6889612545790028
{'learning_rate': 0.03889777777777778, 'n_estimators': 100} - SCORE: 0.6894517752993704
{'learning_rate': 0.07778555555555555, 'n_estimators': 25} - SCORE: 0.6868347105310214
{'learning_rate': 0.07778555555555555, 'n_estimators': 50} - SCORE: 0.6864519434937025
{'learning_rate': 0.07778555555555555, 'n_estimators': 100} - SCORE: 0.6868546920310399
{'learning_rate': 0.11667333333333334, 'n_estimators': 25} - SCORE: 0.68617082542558
{'learning_rate': 0.11667333333333334, 'n_estimators': 50} - SCORE: 0.6879457394560292
{'learning_rate': 0.11667333333333334, 'n_estimators': 100} - SCORE: 0.6894278669640

In [ ]:
model_name = 'CBC'

# models_data[model_name]['model'] = deepcopy(clf)

models_data[model_name]['model'] = CatBoostClassifier(random_state=77, 
                            n_estimators=25, objective='Logloss', eval_metric='F1', verbose=0,
                            learning_rate=0.12, max_depth=10, scale_pos_weight=2.251)
print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

#models_data[model_name]['model'].get_feature_importance()

0.7088034175517102
(0.6, 0.7511940946591403)


#### MLPClassifier

In [ ]:
%%time

model_name = 'MLP'
parametrs = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': np.linspace(1e-5, 1e-3, 5),
    #'max_iter': np.linspace(100, 1000, 10, dtype=int)
}

# models_data[model_name]['model'] = search_parametrs(MLPClassifier(random_state=77), parametrs, X_train, y_train)

models_data[model_name]['model'] = MLPClassifier(activation='identity', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=77, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
print(cross_validate(models_data[model_name]['model']))

update_models_data(model_name)

print(search_best_th([model_name]))

0.6694087911033128
(0.51, 0.6662954197410552)
CPU times: user 4.23 s, sys: 2.89 s, total: 7.12 s
Wall time: 3.87 s


# SOME

In [ ]:
models_data.keys()

dict_keys(['KNC', 'XBC', 'LBC', 'CBC'])

In [ ]:
best_th, best_score = search_best_th(['CBC', 'XBC', 'KNC'])
print(best_th, best_score)

0.44 0.760945474080043


In [ ]:
f1_score(y_train, union_precision(['CBC'], train=True))

0.7787538856602244

# Results

In [ ]:
model = models_data['KNC']['model']

In [ ]:
cross_validate(model)

0.6958448548033224

In [ ]:
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=141, p=2,
                     weights='uniform')

In [ ]:
y = make_precision(model, X_test, test_data)

(16627, 95)
1607455826.1192603_predict.csv
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=141, p=2,
                     weights='uniform')


In [ ]:
make_precision(None, X_test, test_data, y_test=union_precision(['CBC', 'XBC', 'LBC', 'KNC']))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
              gamma=0, learning_rate=0.17, max_delta_step=0, max_depth=12,
              max_leaves=0, min_child_weight=100.0, missing=None,
              n_estimators=25, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=77, reg_alpha=2.0,
              reg_lambda=1.0, scale_pos_weight=2.2464, seed=None, silent=None,
              subsample=0.66, verbosity=1)
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.08000555555555555,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=77,
               reg_alpha=1.0, reg_lambda=0.0,
               scale_pos_weight=

array([1, 0, 0, ..., 1, 1, 1])

In [ ]:
make_precision(None, X_test, test_data, y_test=union_precision_proba(['CBC', 'KNC', 'XBC', 'LBC'], th=best_th, printData=True))

th=0.49
KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
                     metric_params=None, n_jobs=None, n_neighbors=267, p=2,
                     weights='uniform')
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
              gamma=0, learning_rate=0.17, max_delta_step=0, max_depth=12,
              max_leaves=0, min_child_weight=100.0, missing=None,
              n_estimators=25, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=77, reg_alpha=2.0,
              reg_lambda=1.0, scale_pos_weight=2.2464, seed=None, silent=None,
              subsample=0.66, verbosity=1)
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.08000555555555555,
               matric='auc', max_depth=3, max_leaves=0, min_child_samples=20,
               min_child_weight=100.0,

array([1, 0, 0, ..., 1, 1, 1])

In [ ]:
# th=0.34
# KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
#                      metric_params=None, n_jobs=None, n_neighbors=267, p=2,
#                      weights='uniform')
# (16627, 240)
# 1608558363.695832_predict.csv